# Código Scraping TFM

A continuación se muestra el código python utilizado para la recolección de datos de barcos en venta en internet. La página web de la que serán extraídos los datos es https://www.boatinternational.com/yachts-for-sale.

### 1. Abrimos la página web y rechazamos cookies

In [1]:
#Importamos todas las librerias que necesitamos 

import pandas as pd
import sys
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
 
driver = webdriver.Firefox()
driver.get("https://www.boatinternational.com/yachts-for-sale")
time.sleep(3)
driver.maximize_window()

In [2]:
# Rechazamos cookies
cookies = driver.find_element(By.ID, "onetrust-pc-btn-handler")
time.sleep(1)
cookies.click()
time.sleep(3)
cookies = driver.find_element(By.XPATH,"/html/body/div[5]/div[3]/div/div[3]/div[1]/button")
cookies.click()
time.sleep(1)

### 2. Desde la página principal identificamos todos los anuncios de barcos disponibles

In [23]:
# Encontramos todos los elementos
boat_elements = driver.find_elements(By.XPATH, '//div[contains(@class, "card__content-wrapper card__content-wrapper--landscape ")]')
# boat_elements = driver.find_elements(By.XPATH, '//div[contains(@class, "card__title card__title--landscape")]')

In [24]:
len(boat_elements)

788

In [5]:
type(boat_elements)

list

### 3. Creamos un bucle para abrir cada anuncio de venta de barcos en una pestaña nueva, recopilar la información contenida y finalmente cerrarlo para acto seguido repetir el proceso con el siguiente

In [25]:
# Creamos un diccionario y un dataframe vacíos
boat_data = {}
df = pd.DataFrame([boat_data])

In [10]:
boat_data_list = []

In [26]:
# Loop through each boat element
for element in boat_elements:
    try:
        # Extract boat name
        boat_name_element = element.find_elements(By.XPATH, './/div[contains(@class, "card__title--landscape ")]/a')
        boat_name = boat_name_element[0].text.strip() if boat_name_element else None

        # Extract price
        price_elements = element.find_elements(By.XPATH, './/div[contains(@class, "card__details--landscape")]/span[@class=""]')
        price = None
        for price_element in price_elements:
            if any(currency in price_element.text for currency in ["POA", "€", "$", "£"]):
                price = price_element.text.strip()
                break

        # Create a dictionary to store boat data
        boat_data = {
            "Boat Name": boat_name,
            "Price": price
        }

        # Create a DataFrame from the dictionary
        df = pd.concat([df, pd.DataFrame([boat_data])], ignore_index = True)
        boat_data = {}

        # Refresh boat elements to avoid stale references
        boat_elements = driver.find_elements(By.XPATH, '//div[contains(@class, "card__content-wrapper card__content-wrapper--landscape ")]')

    except StaleElementReferenceException:
        print("Element is stale. Skipping to the next element.")


# Display the DataFrame
print(df)

         Boat Name          Price
0              NaN            NaN
1         SHAMANNA          POA •
2             MANA   $6,575,000 •
3        AMARYLLIS  €89,000,000 •
4         GRATEFUL  €15,990,000 •
..             ...            ...
784       VIBRANCE  $21,900,000 •
785  LADY VICTORIA   €1,900,000 •
786       CROSSBOW   €7,950,000 •
787        ECLIPSE   $5,450,000 •
788       GIHRAMAR   €2,280,000 •

[789 rows x 2 columns]


### 4. Guardamos como CSV

In [27]:
import csv

print("Introduzca la ruta en la que quiere guardar el dataframe")
ruta_csv = input("Ruta: ")
# C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2
# Barcos_definitivo_total
print()
print("Ahora escoja un nombre para el archivo")
nombre_archivo_csv = input("Nombre del archivo: ")
print()

df.to_csv(ruta_csv + "/" + nombre_archivo_csv + ".csv", sep = ';')
print("Se grabó la información correctamente")

Introduzca la ruta en la que quiere guardar el dataframe
Ruta: C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2

Ahora escoja un nombre para el archivo
Nombre del archivo: Barcos_precios

Se grabó la información correctamente


### 5. Lo cargamos desde el CSV para comprobar que funciona

In [ ]:
df = pd.read_csv(ruta_csv + "/" + nombre_archivo_csv + ".csv", encoding = "utf-8",
     sep = ";", engine = "python", index_col = 0)
df

### 6. Lo guardamos en formato excel

In [28]:
print("Introduzca la ruta en la que quiere guardar el dataframe")
ruta_excel = input("Ruta: ")
# C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2
# Barcos_definitivo_total
print()
print("Ahora escoja un nombre para el archivo")
nombre_archivo_excel = input("Nombre del archivo: ")
print()
escritor = pd.ExcelWriter(ruta_excel + "/" + nombre_archivo_excel + ".xlsx", engine = 'xlsxwriter')
df.to_excel(escritor, sheet_name="hoja1", index=True)
escritor.save()
print("Se grabó la información correctamente")

Introduzca la ruta en la que quiere guardar el dataframe
Ruta: C:/Users/Usuario/Documents/Python Anaconda Archivos/Almacenes de Datos/Scripts/Scripts parte 2

Ahora escoja un nombre para el archivo
Nombre del archivo: Barcos_precios

Se grabó la información correctamente


### 7. Lo volvemos a cargar para comprobar que funciona

In [ ]:
df = pd.read_excel(ruta_excel + "/" + nombre_archivo_excel + ".xlsx", index_col = 0)
print(df)